In [9]:
import os
import math
from time import perf_counter

import numpy as np
import scipy.io
from scipy.optimize import minimize
from scipy.linalg import cholesky, LinAlgError

import torch
import torch.nn as nn
from torch.nn.utils import parameters_to_vector, vector_to_parameters

torch.set_default_dtype(torch.float64)
torch.manual_seed(2)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

RESULTS_DIR = "results2"
os.makedirs(RESULTS_DIR, exist_ok=True)

In [ ]:
scaling = 1  
L1 = 20
layer_dims = (2, L1, L1, L1, L1, 1)

L = 2
x0 = -1
xf = x0 + L
tfinal = 1
nu = 0.01 / np.pi


Nepochs_ADAM  = 1000
Nchange  = 500
Nint     = 8000
N0       = 500
Nb       = 500
Nprint   = 100

k1 = 1  
k2 = 1  
rad_args = (k1, k2) 


class CustomActivation(nn.Module):
    def __init__(self, **kwargs):
        super().__init__()

    def forward(self, x):
        return x * scaling


def init_variance_scaling_fan_avg_uniform(linear, scale):
    fan_in, fan_out = linear.in_features, linear.out_features
    n = 0.5 * (fan_in + fan_out)
    limit = math.sqrt(3.0 * scale / n)
    with torch.no_grad():
        linear.weight.uniform_(-limit, limit)
        if linear.bias is not None:
            linear.bias.zero_()

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(layer_dims[0], layer_dims[1]), nn.Tanh(),
            nn.Linear(layer_dims[1], layer_dims[2]), nn.Tanh(),
            nn.Linear(layer_dims[2], layer_dims[3]), nn.Tanh(),
            nn.Linear(layer_dims[3], layer_dims[4]), nn.Tanh(),
            nn.Linear(layer_dims[4], layer_dims[5]),
            CustomActivation(),)
        final_linear = self.net[-2]
        init_variance_scaling_fan_avg_uniform(final_linear, scale=1.0/(scaling**2))

    def forward(self, x):
        return self.net(x)

N = Net()

def generate_inputs(Nint):
    t = tfinal * np.random.rand(Nint)
    x = L * np.random.rand(Nint) + x0
    X = np.hstack((t[:, None], x[:, None]))
    print('x', x.shape, 't', t.shape, 'X', X.shape)
    return torch.as_tensor(X)

def initial_points(N0):
    t = np.zeros(N0)
    x = L * np.random.rand(N0) + x0
    X = np.hstack((t[:, None], x[:, None]))
    return torch.as_tensor(X)

def boundary_points(Nb):
    t = tfinal * np.random.rand(Nb)
    x = L * np.random.randint(0, 2, size=(Nb,)) + x0
    X = np.hstack((t[:, None], x[:, None]))
    return torch.as_tensor(X)

def generate_validation(Nt, Nx):
    x = np.linspace(x0, xf, Nx)
    t = np.linspace(0, tfinal, Nt)
    x, t = np.meshgrid(x, t)
    X = np.hstack((t.flatten()[:, None], x.flatten()[:, None]))
    print('x', x.shape, 't', t.shape, 'X', X.shape)
    return torch.as_tensor(X), t, x

def adaptive_rad(N, Nint, rad_args, Nsampling=50000):
    Xtest = generate_inputs(Nsampling)              
    k1, k2 = rad_args
    Y = torch.abs(get_results(N, Xtest)[-1]).reshape(-1)  
    w = (Y**k1)
    err_eq = w / w.mean() + k2
    p = (err_eq / err_eq.sum()).clamp_min(1e-12)     
    X_ids = torch.multinomial(p, num_samples=Nint, replacement=False)
    return Xtest[X_ids]

def uinit(X): 
    x = X[:, 1:2]
    return -torch.sin(torch.pi * x)

def uleft(X): 
    Xleft = X[X[:, 1] == x0]
    t = Xleft[:, 0]
    return torch.zeros((t.shape[0], 1), dtype=torch.float64)

def uright(X):  
    Xright = X[X[:, 1] == xf]
    t = Xright[:, 0]
    return torch.zeros((t.shape[0], 1), dtype=torch.float64)

def output(N, X):
    Nout = N(X)
    u = Nout[:, 0:1]
    return u

def get_results(N, X):
    X.requires_grad_(True)
    u = output(N, X)                 
    grads = torch.autograd.grad(
        u, X,
        grad_outputs=torch.ones_like(u),
        create_graph=True,
        retain_graph=True)[0]        
    u_t = grads[:, 0]                  
    u_x = grads[:, 1]                  

    u_xx = torch.autograd.grad(
        u_x, X,
        grad_outputs=torch.ones_like(u_x),
        create_graph=True,
        retain_graph=True)[0][:, 1]
    fu = u_t + u[:, 0] * u_x - nu * u_xx
    return u, fu


loss_function = nn.MSELoss()
lam0 = 5.0
lamB = 5.0

def loss(fu, u0, u0pinn, ul, ulpinn, ur, urpinn):
    Ntot = fu.shape[0]
    zeros = torch.zeros((Ntot, 1), dtype=torch.get_default_dtype(), device=fu.device)
    fu_col = fu.reshape(-1, 1)
    loss_value = (loss_function(fu_col, zeros)
                  + lam0 * loss_function(u0, u0pinn)
                  + lamB * (loss_function(ul, ulpinn) + loss_function(ur, urpinn)))
    return loss_value

def grads(N, X, X0, Xb):
    for p in N.parameters():
        p.grad = None

    X = X.clone().detach().requires_grad_(True)
    X0 = X0.clone().detach().requires_grad_(True)
    _, fu = get_results(N, X)
    u0 = uinit(X0)
    u0pinn = output(N, X0)
    ul = uleft(Xb)
    ur = uright(Xb)
    mask_l = (Xb[:, 1] == x0)
    mask_r = (Xb[:, 1] == xf)
    ulpinn = output(N, Xb[mask_l])
    urpinn = output(N, Xb[mask_r])

    loss_value = loss(fu, u0, u0pinn, ul, ulpinn, ur, urpinn)

    loss_value.backward()

    gradsN = [p.grad for p in N.parameters()]
    return gradsN, loss_value


def training(N, X, X0, Xb, optimizer):
    _, loss_value = grads(N, X, X0, Xb)
    optimizer.step()
    return loss_value


X  = generate_inputs(Nint)
X0 = initial_points(N0)
Xb = boundary_points(Nb)


template = 'Epoch {}, loss: {}'

rel_adam = [] 
optimizer = torch.optim.Adam(N.parameters(), lr=5e-3, betas=(0.99, 0.999), eps=1e-20)
adam_scheduler = torch.optim.lr_scheduler.LambdaLR(
    optimizer, lr_lambda=lambda step: 0.98 ** (step / 1000.0))

adam_losses = []
start_time = perf_counter()

adam_t0 = perf_counter()
for epoch in range(Nepochs_ADAM):
    if (epoch + 1) % Nchange == 0:
        X  = adaptive_rad(N, Nint, rad_args)
        X0 = initial_points(N0)
        Xb = boundary_points(Nb)

    loss_value = training(N, X, X0, Xb, optimizer)
    adam_losses.append(float(loss_value.detach().cpu().item()))

    if (epoch + 1) % Nprint == 0:
        print(template.format(epoch + 1, adam_losses[-1]))

        N.eval()
        with torch.no_grad():
            u_pred = output(N, X_star).detach().cpu().numpy().reshape(u_ref.shape)
        N.train()

        num = np.linalg.norm(u_ref - u_pred)
        den = np.linalg.norm(u_pred) + 1e-12
        rel = num / den
        rel_adam.append((epoch + 1, float(rel)))


adam_time_sec = perf_counter() - adam_t0

def nested_tensor(grad, layer_dims, train_activations=False, bias=True):
    if _has_torch and isinstance(grad, torch.Tensor):
        grad = grad.detach().cpu().numpy()
    grad = np.asarray(grad).ravel()

    if not train_activations:
        if bias:
            temp = [None] * (2 * len(layer_dims) - 2)  
        else:
            temp = [None] * (2 * len(layer_dims) - 3) 

        index = 0
        for i in range(len(temp)):
            if i % 2 == 0:
                k = i // 2
                fan_in, fan_out = layer_dims[k], layer_dims[k + 1]
                expected_shape = (fan_in, fan_out)
                size = fan_in * fan_out

                print('layer_dims', layer_dims)
                print(f"Expected shape: {expected_shape}, Grad slice size: {size}")
                print(f"Current index: {index}, Next index: {index + size}")
                print(f"Size of grad slice: {grad[index:index+size].size}")

                temp[i] = grad[index:index + size].reshape(expected_shape)
                index += size
            else:
                k = i - (i // 2)       
                bsz = layer_dims[k]
                temp[i] = grad[index:index + bsz]
                index += bsz
        return temp

    else:
        temp = [None] * (3 * len(layer_dims) - 4)
        index = 0
        for i in range(len(temp)):
            if i % 3 == 0:
          
                k = i // 3
                fan_in, fan_out = layer_dims[k], layer_dims[k + 1]
                size = fan_in * fan_out
                temp[i] = grad[index:index + size].reshape(fan_in, fan_out)
                index += size
            elif i % 3 == 1:

                k = int((i + 2) / 3)  
                bsz = layer_dims[k]
                temp[i] = grad[index:index + bsz]
                index += bsz
            else:
  
                temp[i] = grad[index]
                index += 1
        return temp


power = 1.0  

def loss_and_gradient_torch(N, X, X0, Xb, power=power):
    X = X.clone().detach().requires_grad_(True)
    _, fu = get_results(N, X)
    u0 = uinit(X0)
    u0pinn = output(N, X0)
    ul = uleft(Xb)
    mask_l = (Xb[:, 1] == x0)
    ulpinn = output(N, Xb[mask_l])
    ur = uright(Xb)
    mask_r = (Xb[:, 1] == xf)
    urpinn = output(N, Xb[mask_r])
    loss_value = loss(fu, u0, u0pinn, ul, ulpinn, ur, urpinn)
    loss_root = loss_value if power == 1.0 else loss_value ** (1.0 / power)
    params = list(N.parameters())
    gradsN = torch.autograd.grad(
        loss_root, params,
        create_graph=False, retain_graph=False, allow_unused=False)
    return loss_root, gradsN


def loss_and_gradient(weights, N, X, X0, Xb, layer_dims=None):
    first_param = next(N.parameters())
    device = first_param.device
    dtype = first_param.dtype
    w_tensor = torch.as_tensor(weights, dtype=dtype, device=device)
    
    with torch.no_grad():
        vector_to_parameters(w_tensor, N.parameters())
    loss_val, grads_list = loss_and_gradient_torch(N, X, X0, Xb, power=power)
    grads_flat = torch.cat([g.reshape(-1) for g in grads_list])

    return float(loss_val.detach().cpu().item()), grads_flat.detach().cpu().numpy()


Nbfgs = 10000
Nbatches = int(round(Nbfgs / Nchange))
Nprint = 100
n_ckpts        = Nbfgs // Nprint                  
warmup_steps   = len(adam_losses)                  
epochs_bfgs    = warmup_steps + np.arange(1, n_ckpts + 1, dtype=float) * Nprint
lossbfgs        = np.zeros(n_ckpts, dtype=float)
validation_list = np.zeros(n_ckpts, dtype=float)
error_list      = np.zeros(n_ckpts, dtype=float)
time_elapsed = np.array([])  
initial_time = perf_counter()

Nt = 300
Nx = 300
Xtest, t, x = generate_validation(Nt, Nx)
X0test = Xtest[:Nx]
mask_b = (Xtest[:, 1] == x0) | (Xtest[:, 1] == xf)
Xbtest = Xtest[mask_b]

mat = scipy.io.loadmat("burgers_canonical.mat")
u_ref  = mat["usol"]
t_star = mat["t"].ravel()
x_star = mat["x"].ravel()
x_star, t_star = np.meshgrid(x_star, t_star)

device = next(N.parameters()).device
X_star = torch.as_tensor(
    np.hstack((t_star.reshape(-1,1), x_star.reshape(-1,1))),
    dtype=torch.get_default_dtype(),
    device=device)


initial_weights = parameters_to_vector([p.detach() for p in N.parameters()]).cpu().numpy()

cont = 0
def callback(*, intermediate_result):
    global N, cont, lossbfgs, Nprint, u_ref, \
           x_star, X_star, Xtest, validation_list, X0test, Xbtest, error_list, power

    device = next(N.parameters()).device
    dtype  = next(N.parameters()).dtype

    cont += 1
    if (cont % Nprint) != 0:
        return

    idx = cont // Nprint - 1
    if idx < 0 or idx >= len(lossbfgs):  # guard
        return

    loss_value = float((intermediate_result.fun) ** power)
    lossbfgs[idx] = loss_value

    _, futest = get_results(N, Xtest.to(device))
    zeros = torch.zeros((futest.shape[0], 1), dtype=dtype, device=device)

    with torch.no_grad():
        u_pred = output(N, X_star).reshape(x_star.shape)        
        u_ref_t = torch.as_tensor(u_ref, dtype=dtype, device=device)
        v1 = loss_function(u_pred, u_ref_t)
        v2 = loss_function(futest.reshape(-1,1), zeros)
        validation_list[idx] = (v1 + v2).item()

    u_np = u_pred.detach().cpu().numpy()
    num = np.linalg.norm(u_ref - u_np)
    den = np.linalg.norm(u_np) + 1e-12
    error_list[idx] = num / den

    print(f"Iteration {cont} (ckpt {idx+1}/{len(lossbfgs)}): "
          f"Train {loss_value:.3e}, relL2 {error_list[idx]:.3e}")


method = "BFGS"
method_bfgs = "SSBroyden2"      
initial_scale = False      
warmup_tag = "warmup_adam" 

os.makedirs("results2", exist_ok=True)

def _scipy_cb_factory():
    class _Res:
        __slots__ = ("fun",)
        def __init__(self, fun): self.fun = fun
    def _cb(xk):
        f, _ = loss_and_gradient(xk, N, X, X0, Xb, layer_dims)
        callback(intermediate_result=_Res(f))
    return _cb

H0 = np.eye(initial_weights.size, dtype=np.float64)

initial_time_bfgs = perf_counter()

bfgs_loss_ckpt = []
bfgs_rel       = []

bfgs_t0 = perf_counter()

while cont < Nbfgs: 
    print(cont)
    result = minimize(
        loss_and_gradient, 
        initial_weights, 
        args=(N, X, X0, Xb, layer_dims),
        method=method,
        jac=True,
        options={
            'maxiter': Nchange,
            'gtol': 0,
            'hess_inv0': H0,          
            'method_bfgs': method_bfgs,
            'initial_scale': initial_scale },
        tol=0,
        callback=callback) 

    initial_weights = result.x
    H0 = result.hess_inv
    H0 = 0.5 * (H0 + H0.T)
    
    try:
        cholesky(H0)
    except LinAlgError:
        H0 = np.eye(len(initial_weights), dtype=np.float64) 

    X  = adaptive_rad(N, Nint, rad_args)
    X0 = initial_points(N0)
    Xb = boundary_points(Nb)
    _, fu   = get_results(N, X)
    u0      = uinit(X0)
    u0pinn  = output(N, X0)
    ul      = uleft(Xb)
    ulpinn  = output(N, Xb[Xb[:, 1] == x0])
    ur      = uright(Xb)
    urpinn  = output(N, Xb[Xb[:, 1] == xf])
    loss_value = loss(fu, u0, u0pinn, ul, ulpinn, ur, urpinn)

    initial_scale = False

bfgs_time_sec = perf_counter() - bfgs_t0